# 01 Import libraries

In [4]:
import numpy as np
import torch


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [5]:
import pandas as pd

In [8]:
from scipy.optimize import minimize

# 02 Load data

In [9]:
# file path for Colab. May need to change this
decomp_cov_df = pd.read_csv('decomp_cov.csv')
mean_df = pd.read_csv('mean.csv')

In [10]:
decomp_cov_np = np.array(decomp_cov_df)
mean_np = np.array(mean_df).squeeze()

# 03 Compute the approximate optimality for the stochastic objective

We evaluate the optimality value for the ERM style objective

$$E[{h(\theta^*(\tilde\lambda), \tilde\lambda)}] \simeq \frac{1}{1000} \sum_{i=1}^{1000} h(\theta^*(\lambda_i), \lambda_i)$$

In [11]:
dim_lambda = 12
lam_max_high_dim = np.ones(dim_lambda)
lam_min_high_dim = np.zeros(dim_lambda)
lam_min_high_dim[2:] = -2
lam_min_high_dim[0] = .2
lam_max_high_dim[2:] = 2
input_dim = decomp_cov_np.shape[1]

In [12]:
from scipy.stats import uniform

In [13]:
np.random.seed(42)
train_set = []
for j in range(1000):
    samples = uniform.rvs(loc=0, scale=1, size=len(lam_min_high_dim))
    train_set.append(samples * (lam_max_high_dim - lam_min_high_dim) + lam_min_high_dim)

In [14]:
train_set[0]

array([ 0.4996321 ,  0.95071431,  0.92797577,  0.39463394, -1.37592544,
       -1.37602192, -1.76766555,  1.46470458,  0.40446005,  0.83229031,
       -1.91766202,  1.87963941])

In [15]:
np.random.seed(88)
val_set = []
for j in range(1000):
    samples = uniform.rvs(loc=0, scale=1, size=len(lam_min_high_dim))
    val_set.append(samples * (lam_max_high_dim - lam_min_high_dim) + lam_min_high_dim)

In [16]:
# Define the objective function
def objective(theta, Sigma, mu, const, hyper_params):
    # print(Sigma, mu, const, hyper_params)
    quadratic_term = hyper_params[0] * (theta.T @ Sigma @ theta)
    linear_term = -hyper_params[1] * (mu.T @ theta)
    # regularizer = np.sum(np.sqrt(theta**2 + const**2) - const)
    regularizer = np.sum((theta - hyper_params[2:])**2)
    
    return quadratic_term + linear_term + regularizer

# Define the equality constraint: sum(theta) = 1
def constraint(theta):
    return np.sum(theta) - 1

In [17]:
const = .01

# Initial guess for theta
initial_theta = np.zeros(input_dim)

cov = decomp_cov_np.T @ decomp_cov_np

# Define the constraint in a dictionary format for the 'minimize' function
# cons = {'type': 'eq', 'fun': constraint}

In [18]:
cov

array([[ 14.88694787,  17.96342851,  15.62749789,  18.15608029,
         12.8475343 ,  13.45480821,  12.39359279,  10.4856145 ,
         10.77197552,  15.05393296],
       [ 17.96342851, 114.30831417,  32.4917756 ,  36.40362025,
         40.72530978,  22.52463238,  35.4337486 ,  21.24320714,
         11.91429053,  35.77522809],
       [ 15.62749789,  32.4917756 ,  25.62290251,  28.33939666,
         21.19279383,  18.61550578,  19.31985585,  16.13689191,
         12.23087011,  24.51854274],
       [ 18.15608029,  36.40362025,  28.33939666,  77.09328733,
         19.46612271,  24.41378517,  18.68226607,  16.41267805,
         13.30689891,  31.76519195],
       [ 12.8475343 ,  40.72530978,  21.19279383,  19.46612271,
         29.886921  ,  16.34515817,  22.70028218,  16.1628709 ,
         10.16682333,  21.53193377],
       [ 13.45480821,  22.52463238,  18.61550578,  24.41378517,
         16.34515817,  23.72400703,  15.24448859,  12.99003046,
          9.9165135 ,  19.83319497],
       [ 1

In [19]:
np.linalg.eig(cov)

EigResult(eigenvalues=array([236.68923798,  61.45897145,  33.43222483,  12.1784997 ,
         8.94740675,   1.81553943,   3.34613198,   3.58231072,
         6.56783868,   5.47372845]), eigenvectors=array([[ 0.18519845, -0.13262567,  0.23054935,  0.27880374,  0.13515215,
        -0.20641795,  0.51652365, -0.67739051, -0.13116235,  0.11616721],
       [ 0.58523094,  0.67236705, -0.38324527,  0.16267429,  0.1094294 ,
         0.01773662,  0.03351783,  0.01432152, -0.10964003, -0.08030304],
       [ 0.29310401, -0.12746647,  0.22019026, -0.00861423,  0.07398746,
         0.74227726,  0.00150766,  0.0854623 , -0.16606627,  0.50788839],
       [ 0.40476037, -0.65273657, -0.59024754, -0.03076027, -0.20049573,
         0.02188337,  0.02510892, -0.03568027,  0.06723045, -0.1168964 ],
       [ 0.29801838,  0.10822257,  0.29308921, -0.25865215, -0.39263761,
        -0.07142364,  0.44258068,  0.22801069,  0.58159914,  0.03579076],
       [ 0.23551478, -0.18531653,  0.23774224, -0.05359831,  0.7336

In [21]:
weights = []
out = 0.0
losses = []
for i in range(len(val_set)):
    hyper_params = val_set[i]
    # Run the optimization using 'trust-constr' method
    result = minimize(
        fun=objective,  # Objective function
        x0=initial_theta,  # Initial guess for theta
        args=(cov, mean_np, const, hyper_params),  # Arguments passed to the objective function
        method='trust-constr',  # Trust-region optimization method
        # constraints=cons  # Constraints
    )
    if i % 100 == 0:
        print(f"first {i} results found")
    weights.append(result.x)
    out += result.fun
    losses.append(result.fun)
print(f'validation set loss = {out/len(val_set)}')

first 0 results found
first 100 results found
first 200 results found
first 300 results found
first 400 results found
first 500 results found
first 600 results found
first 700 results found
first 800 results found
first 900 results found
validation set loss = 10.414102515751397


In [18]:
# weights = []
out = 0.0
for i in range(len(train_set)):
    hyper_params = train_set[i]
    # Run the optimization using 'trust-constr' method
    result = minimize(
        fun=objective,  # Objective function
        x0=initial_theta,  # Initial guess for theta
        args=(cov, mean_np, const, hyper_params),  # Arguments passed to the objective function
        method='trust-constr',  # Trust-region optimization method
        # constraints=cons  # Constraints
    )
    if i % 100 == 0:
        print(f"first {i} results found")
    # weights.append(result.x)
    out += result.fun
print(f'training set loss = {out/len(train_set)}')

first 0 results found
first 100 results found
first 200 results found
first 300 results found
first 400 results found
first 500 results found
first 600 results found


c:\Users\dongq\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


first 700 results found
first 800 results found
first 900 results found
training set loss = 10.428659738259746
